In [ ]:
# Install dependencies
!pip install -q datasets transformers accelerate peft trl bitsandbytes fastapi uvicorn fsspec==2025.3.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
# Load dataset
dataset = load_dataset("b-mc2/sql-create-context", split='train')

# Preprocessing
def preprocess(example):
    input_text = "Translate to SQL: " + example["question"]
    output_text = example["query"]
    return {"input_text": input_text, "output_text": output_text}

dataset = dataset.map(preprocess)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
# Tokenizer
model_name = "meta-llama/Llama-2-7b-hf"  # Replace with local model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize(example):
    return tokenizer(
        example["input_text"], text_target=example["output_text"],
        padding="max_length", truncation=True, max_length=512
    )

tokenized_dataset = dataset.map(tokenize, batched=True)


In [ ]:
# Load model with LoRA
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


In [ ]:
# Save model
model.save_pretrained("fine-tuned-llama-sql")
tokenizer.save_pretrained("fine-tuned-llama-sql")

# REST API using FastAPI
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()
model = AutoModelForCausalLM.from_pretrained("fine-tuned-llama-sql")
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-llama-sql")

class QueryInput(BaseModel):
    question: str

@app.post("/sql")
def generate_sql(query: QueryInput):
    prompt = "Translate to SQL: " + query.question
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=128)
    sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"sql_query": sql}
